In [32]:
# testing text processing and possible rules to implement

import pandas as pd
# import nltk 
from nltk import tokenize #, word_tokenize
from nltk.corpus import stopwords
# from spacy.morphology import Morphology
# import spacy
# nlp = spacy.load("en_core_web_sm")

# nlp = spacy.load("en_core_web_sm")
# from nltk.parse.stanford import StanfordParser
# from pycorenlp import StanfordCoreNLP


In [29]:
# data will be a list of strings from dataframe
smoker_data_df = pd.read_csv('smoker_status.csv')

identifiers = smoker_data_df['row_id'].to_list()
statuses = smoker_data_df['status'].to_list()
med_notes = smoker_data_df['text'].to_list()

assert len(identifiers) == len(statuses) == len(med_notes), 'Three cols not the same lengths.'

In [13]:
# split into sentences test
test_row = smoker_data_df.iloc[[30]].values.tolist()
test_sent = test_row[0][2]
print(test_row)

# split into sentences
split_sent = tokenize.sent_tokenize(test_sent)
print('--------------')
print('sentence split')
print(split_sent)
# not effective for most of medical notes, returns full sentences

[[34464, 'Smoker', "Past Medical History: current smoker  Social History: married current smoker >10cigrettes/day current ETOH use,heavy per patient's family  Family History: + father and brother died of myocardial infraction  Physical Exam: Vitals:97.9-63-19"]]
--------------
sentence split
["Past Medical History: current smoker  Social History: married current smoker >10cigrettes/day current ETOH use,heavy per patient's family  Family History: + father and brother died of myocardial infraction  Physical Exam: Vitals:97.9-63-19"]


In [15]:
# split into words test (most effective would have been sentences then words)
wordtok_sent = nltk.word_tokenize(test_sent)
print(wordtok_sent)

['Past', 'Medical', 'History', ':', 'current', 'smoker', 'Social', 'History', ':', 'married', 'current', 'smoker', '>', '10cigrettes/day', 'current', 'ETOH', 'use', ',', 'heavy', 'per', 'patient', "'s", 'family', 'Family', 'History', ':', '+', 'father', 'and', 'brother', 'died', 'of', 'myocardial', 'infraction', 'Physical', 'Exam', ':', 'Vitals:97.9-63-19']


In [16]:
# split all sentences into words
split_words = [nltk.word_tokenize(every_text)for every_text in med_notes]

In [17]:
# create baseline
    
# if words not related to smoking does not appear, label as unknown ==> F1
smoke_words = ['smoke', 
               'smoker', 
               'smoking', 'Smoking',
               'smokes',
               'smoked', 'Smoked', 
               'ppd', # maybe not needed
               'nonsmoker', 'Nonsmoker',]
                # useful words to add: 'tob', 'tobacco', 'cigs', 'cigarettes'


# all sentences should be okay for this dataset
def baseline(all_sents):
    
    to_label_unknown = []
    
    for every_sentence_idx in range(len(all_sents)):
        
        word_list = all_sents[every_sentence_idx]
        
        # check if any of the smoke_words appear in each sentence
        if any([every_smoke_word in word_list for every_smoke_word in smoke_words]) == False:
            
            to_label_unknown.append(every_sentence_idx)
    
    return to_label_unknown
    
#     return to_unknown

baseline(split_words)
    
    

[22]

In [43]:
# next state, nonsmoker
    # labels have 'nonsmoker' or 'Nonsmoker'
    # or negation, POS?

# all_sents = after baseline
def find_nonsmokers(all_sents):
    
    nonsmoker_words = ['nonsmoker', 'Nonsmoker']
    
    to_label_nonsmoker = []
    
    for every_sentence_idx in range(len(all_sents)):
        
        word_list = all_sents[every_sentence_idx]
        
        # check if any of nonsmoker_words appear in each sentence
        if any([every_word in word_list for every_word in nonsmoker_words]) == True:
            
            to_label_nonsmoker.append(every_sentence_idx)
        
        
        # check if text includes 'does not smoke', 'no h/o smoking', 'does not drink, smoke, or do any drugs'
            # relies on text only having 'smoker' appear once
            
        # 'no h/o smoking'
        if word_list.count('smoking') == 1:
            
            #if 'smoking' in word_list:

            smoking_idx = word_list.index('smoking')
            ho_idx = smoking_idx - 1
            no_idx = smoking_idx - 2

            # 'no', 'h/o', 'smoking' must appear in exact order in word list
            if word_list[ho_idx] == 'h/o' and word_list[no_idx] == 'no':

                to_label_nonsmoker.append(every_sentence_idx)
                #print(word_list)
        
        # 'does not smoke'
        if word_list.count('smoke') == 1:
            
            #if 'smoke' in word_list:
            smoke_idx = word_list.index('smoke')
            not_idx = smoke_idx - 1 
            does_idx = smoke_idx - 2

            if word_list[not_idx] == 'not' and word_list[does_idx] == 'does':

                to_label_nonsmoker.append(every_sentence_idx)
        
        
    print('done all sents')
    print(to_label_nonsmoker)
#     return to_label_nonsmoker 

find_nonsmokers(split_words)     
# [15, 16, 27, 37, 38, 55, 65]

done all sents
[15, 16, 27, 37, 38, 55, 65]


In [41]:
# next state, unknown labels in data all have 'with a smoker in the household'
    # too specific, should have another check in case it has other?

def smoker_in_household(all_sents):
    
    to_label_unknown = []
    
    for every_sentence_idx in range(len(all_sents)):
        
        word_list = all_sents[every_sentence_idx]
        
        # check if 'smoker in the household' is in sentence
            # relies on text only having 'smoker' appear once
        if word_list.count('smoker') == 1:
            
            #if 'smoker' in word_list:

            smoker_idx = word_list.index('smoker')
            in_idx = smoker_idx + 1
            the_idx = smoker_idx + 2
            household_idx = smoker_idx + 3

            # 'smoker', 'in', 'the', 'household' must appear in exact order in word list
            if word_list[in_idx] == 'in' and word_list[the_idx] == 'the' and word_list[household_idx] == 'household':
                to_label_unknown.append(every_sentence_idx)

            
    print('done sents')
    print(to_label_unknown)
    
        
smoker_in_household(split_words)

done sents
[0, 4, 17, 28, 32, 34, 59, 61, 67]


In [48]:
# after smoker words inside word_list

# check suffixes of words in sentences that have not been labelled as unknown
    # --> should diverge to Smoker and Former Smoker labels



TypeError: Argument 'strings' has incorrect type (expected spacy.strings.StringStore, got str)

In [97]:
# find former smokers

# med_notes are not split into words

def find_former_smokers(all_sents):
    
    to_label_formersmoker = []
    temporal_words = ['years', 'months', 'weeks', 'days']#['years', 'yrs', 'months', 'weeks', 'days']
    approximate_words = ['under', 'over', 'around']
    
    
    for every_sentence_idx in range(len(all_sents)):
        
        word_list = all_sents[every_sentence_idx]

        pos_tagged = nltk.pos_tag(word_list)
#         print('word list')
#         print(word_list)
        
#         print('pos-tagged')
#         print(pos_tagged)

        
        if word_list.count('quit') == 1 or word_list.count('Quit') == 1:
            
            lowercase_quit = [every_word.lower() if every_word == 'Quit' else every_word for every_word in word_list]
#             print(lowercase_quit)
            
            quit_idx = lowercase_quit.index('quit')
            after_quit_idx = quit_idx + 1


            # check if 'quit smoking A_NUMBER TEMPORAL_WORD ago'
            if word_list[after_quit_idx] == 'smoking' and pos_tagged[after_quit_idx + 1][1] == 'CD' and word_list[after_quit_idx + 2] in temporal_words and word_list[after_quit_idx + 3] == 'ago':
                print('quit smoking # time ago')
                print(word_list)
                to_label_formersmoker.append(every_sentence_idx)
            
            # takes into account 'quit smoking X A_NUMBER TEMPORAL_WORD ago' X = over, around, under
            if word_list[after_quit_idx] == 'smoking' and word_list[after_quit_idx + 1] in approximate_words and pos_tagged[after_quit_idx + 2][1] == 'CD' and word_list[after_quit_idx + 3] in temporal_words and word_list[after_quit_idx + 4] == 'ago':
                print('quit smoking X A_NUMBER TEMPORAL_WORD ago')
                print(word_list)
                to_label_formersmoker.append(every_sentence_idx)
                
                
            # might indicate that quit something other than smoking
            if pos_tagged[after_quit_idx][1] == 'CD' and word_list[after_quit_idx + 1] in temporal_words and word_list[after_quit_idx + 2] == 'ago':
                print('quit # time ago')
                print(word_list)
                to_label_formersmoker.append(every_sentence_idx)
        
        # search 'former smoker'
        if word_list.count('smoker') == 1:
            smoker_idx = word_list.index('smoker')
            before_smoker_idx = smoker_idx - 1
            
            # allows 'former' and 'Former'
            if word_list[before_smoker_idx].lower() == 'former':
                
                # some texts have both 'quit smoking X years ago' AND 'former smoker'
                if every_sentence_idx not in to_label_formersmoker:
                    to_label_formersmoker.append(every_sentence_idx)
                

    print('done sentences')
    print(to_label_formersmoker)

find_former_smokers(split_words)
# [1, 2, 3, 19, 23, 35, 47, 49, 60]

quit smoking # time ago
['Quit', 'smoking', '23', 'years', 'ago', '.']
quit # time ago
['She', 'smoked', 'two', 'packs', 'per', 'day', 'for', '40', 'years', 'and', 'quit', 'four', 'years', 'ago', '.']
quit # time ago
['Former', 'smoker', ',', 'quit', '20', 'years', 'ago', '.']
quit smoking X A_NUMBER TEMPORAL_WORD ago
['He', 'quit', 'smoking', 'over', '15', 'years', 'ago', '.']
quit smoking # time ago
['He', 'is', 'a', 'retired', 'accountant', 'who', 'quit', 'smoking', '20', 'years', 'ago', '.']
quit smoking # time ago
['Per', 'Dr.', '*', '*', 'Name', '(', 'NI', ')', '105962', '*', '*', 'note', 'in', 'OMR', ':', 'He', 'quit', 'smoking', '25', 'years', 'ago', '.']
quit # time ago
['He', 'has', 'a', '30pk', 'year', 'smoking', 'hx', 'but', 'quit', '10', 'years', 'ago', '.']
quit # time ago
['Former', 'smoker', 'but', 'quit', '30', 'years', 'ago', '.']
quit smoking # time ago
['Quit', 'smoking', '25', 'years', 'ago', '.']
done sentences
[1, 2, 3, 12, 19, 23, 35, 47, 49, 60]


In [92]:
nltk.pos_tag(split_words[12])

[('He', 'PRP'),
 ('quit', 'VBD'),
 ('smoking', 'VBG'),
 ('over', 'IN'),
 ('15', 'CD'),
 ('years', 'NNS'),
 ('ago', 'RB'),
 ('.', '.')]

In [93]:
split_words[12]

['He', 'quit', 'smoking', 'over', '15', 'years', 'ago', '.']

In [96]:
nltk.pos_tag(['He', 'quit', 'smoking', 'around', '15', 'years', 'ago', '.'])

[('He', 'PRP'),
 ('quit', 'VBD'),
 ('smoking', 'VBG'),
 ('around', 'RB'),
 ('15', 'CD'),
 ('years', 'NNS'),
 ('ago', 'RB'),
 ('.', '.')]

In [35]:
# stopwords

# en_stop_words = stopwords.words('english')
# en_stop_words

In [ ]:
# will have to compare if indices appear twice in the "to_label_x" lists

# indices that have no labels are unknown (initialise all unknown then replace with statuses?)